<a href="https://colab.research.google.com/github/hahw94/DNN-ResNet-etc/blob/master/NeedToOrganize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import pandas as pd

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from skorch import NeuralNetRegressor

from sklearn.model_selection import cross_val_predict

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

torch.manual_seed(0);

import numpy as np
import torch
from skorch import NeuralNetBinaryClassifier
from skorch.dataset import CVSplit
from skorch.callbacks import EarlyStopping
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

In [ ]:
def read_dataset():
    interpolate_data = pd.read_csv("PM10_111121.0_interpolate.csv")
    interpolate_data.head()
    temp_x = interpolate_data[interpolate_data.columns.difference(['Unnamed: 0','PM10'])]
    temp_y = interpolate_data["PM10"]
    X = np.array(temp_x, dtype=np.float32)
    y = np.array(temp_y, dtype=np.float32)
    print(X.shape, y.shape)
    y = np.expand_dims(y, axis=1)
    print(X.shape, y.shape)
    train_x, test_x, train_y, test_y = train_test_split(X,y,train_size = 0.7, test_size = 0.3)
    print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)
    print(train_x.shape, train_y.shape)
    type(train_x[0][0]), type(train_y[0][0])
    return train_x, test_x, train_y, test_y

In [ ]:
train_x, test_x, train_y, test_y = read_dataset()
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(22794, 13) (22794,)
(22794, 13) (22794, 1)
(15955, 13) (15955, 1) (6839, 13) (6839, 1)
(15955, 13) (15955, 1)
(15955, 13) (15955, 1) (6839, 13) (6839, 1)


# (SKORCH) 1. DNN Model

In [ ]:
class DNN_Network(nn.Module):
    def __init__(self):
        super(DNN_Network, self).__init__()
        self.fc1 = nn.Linear(13, 300)
        self.fc2 = nn.Linear(300, 300)
        self.fc3 = nn.Linear(300, 100)
        self.fc4 = nn.Linear(100, 20)
        self.output = nn.Linear(20, 1)
        self.dropout = 0.1
        self.dropout = nn.Dropout(0.2)

    def forward(self, x, **kwargs):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.output(x)
        return x

# 1번째 모델

net = NeuralNetRegressor(
    DNN_Network,
    max_epochs = 100,
    lr = 0.0001,
    optimizer__momentum=0.5,
    batch_size = 32,
    callbacks=[EarlyStopping(patience=50)],
    device = 'cuda'
)

- 100번 학습 진행해본결과 0.78(테스트데이터기준)의 R값을 보여줌 / 0.897(학습데이터)

# 2번째 모델

net = NeuralNetRegressor(
    DNN_Network,
    max_epochs = 200,
    lr = 0.00005,
    optimizer__momentum=0.5,
    batch_size = 32,
    callbacks=[EarlyStopping(patience=50)],
    device = 'cuda'
)

    195       12.8560      294.3912  2.4009
    196       11.2754      288.8933  2.3587
    197       11.1229      288.4763  2.1982
    198       10.7868      295.6834  2.1743
    199       10.2079      288.4824  2.2799
    200        9.3126      291.0469  2.1772
    
    
            Pred	Target
Pred	1.000000	0.792919
Target	0.792919	1.000000

          Train	Target
Train	1.00000	0.95392
Target	0.95392	1.00000

# 3번째 모델

net = NeuralNetRegressor(
    DNN_Network,
    max_epochs = 500,
    lr = 0.00005,
    optimizer__momentum=0.8,
    batch_size = 64,
    #callbacks=[EarlyStopping(patience=50)],
    device = 'cuda'
)

    496        1.7015      279.3411  1.2124
    497        1.6967      279.0720  1.1793
    498        1.6932      278.9402  1.1709
    499        1.7029      279.0405  1.1574
    500        1.6867      279.4282  1.1684
    
Pred	Target
Pred	1.000000	0.800654
Target	0.800654	1.000000

Train	Target
Train	1.000000	0.963562
Target	0.963562	1.000000

# 4번째 모델

net = NeuralNetRegressor(
    DNN_Network,
    max_epochs = 2000,
    lr = 0.00001,
    optimizer__momentum=0.9,
    batch_size = 64,
    #callbacks=[EarlyStopping(patience=50)],
    device = 'cuda'
)

   1998        0.6039      278.6450  1.0891
   1999        0.6034      278.7747  1.1290
   2000        0.6031      278.9720  1.1309
<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=DNN_Network(
    (fc1): Linear(in_features=13, out_features=300, bias=True)
    (fc2): Linear(in_features=300, out_features=300, bias=True)
    (fc3): Linear(in_features=300, out_features=100, bias=True)
    (fc4): Linear(in_features=100, out_features=20, bias=True)
    (output): Linear(in_features=20, out_features=1, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  ),
)


	Pred	Target
Pred	1.000000	0.803782
Target	0.803782	1.000000

Train	Target
Train	1.000000	0.965438
Target	0.965438	1.000000

# 5번째 모델

class DNN_Network(nn.Module):
    def __init__(self):
        super(DNN_Network, self).__init__()
        self.fc1 = nn.Linear(13, 500)
        self.fc2 = nn.Linear(500, 500)
        self.fc3 = nn.Linear(500, 100)
        self.fc4 = nn.Linear(100, 20)
        self.output = nn.Linear(20, 1)
        self.dropout = 0.1
        self.dropout = nn.Dropout(0.3)

    def forward(self, x, **kwargs):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.output(x)
        return x

    499        3.5337      267.4550  0.7021
    500        3.5313      265.1248  0.7302
<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=DNN_Network(
    (fc1): Linear(in_features=13, out_features=500, bias=True)
    (fc2): Linear(in_features=500, out_features=500, bias=True)
    (fc3): Linear(in_features=500, out_features=100, bias=True)
    (fc4): Linear(in_features=100, out_features=20, bias=True)
    (output): Linear(in_features=20, out_features=1, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
  ),
)

Pred	Target
Pred	1.000000	0.815819
Target	0.815819	1.000000

	Train	Target
Train	1.000000	0.966104
Target	0.966104	1.000000

# (SKORCH) 2. AutoEncoder(Residual Network)

In [ ]:
class DNN_Network(nn.Module):
    def __init__(self):
        super(DNN_Network, self).__init__()
        self.encode1 = nn.Linear(13, 96)
        #self.encode2 = nn.Linear(128, 96)
        self.encode3 = nn.Linear(96, 64)
        self.encode4 = nn.Linear(64, 32)
        self.last_encode = nn.Linear(32, 16)
        #self.latent_layer = nn.Linear(16,16)
        self.decode4 = nn.Linear(16, 32)
        self.decode3 = nn.Linear(32, 64)
        #self.decode2 = nn.Linear(64, 96)
        self.decode1 = nn.Linear(64, 96)
        self.last_decode = nn.Linear(96, 13)
        self.output = nn.Linear(13, 1)
        self.dropout = 0.1
        self.dropout = nn.Dropout(0.3)

    def forward(self, x, **kwargs):
        encode1 = F.relu(self.encode1(x))
        #encode2 = F.relu(self.encode2(encode1))
        encode3 = F.relu(self.encode3(encode1))
        encode4 = F.relu(self.encode4(encode3))
        last_encode = F.relu(self.last_encode(encode4))
        #latent_layer = F.relu(self.latent_layer(last_encode))
        decode4 = F.relu(self.decode4(last_encode)  + encode4)
        decode3 = F.relu(self.decode3(decode4) + encode3)
        #decode2 = F.relu(self.decode2(decode3) + encode2)
        decode1 = F.relu(self.decode1(decode3) + encode1)
        last_decode = F.relu(self.last_decode(decode1) + x)
        output = self.output(last_decode)
        return output

In [ ]:
net = NeuralNetRegressor(
    DNN_Network,
    max_epochs = 300,
    lr = 0.004,
    optimizer=optim.Adam,
    iterator_train__shuffle=True,
    iterator_train__num_workers=4,
    iterator_valid__shuffle=True,
    iterator_valid__num_workers=4,
    #optimizer__momentum = 0.5,
    batch_size = 128,
    #train_split = ,
    #callbacks=[EarlyStopping(patience=50)],
    device = 'cuda'
)

In [ ]:
net.fit(train_x, train_y)

# (SKORCH) 2. AutoEncoder(Residual Network) + Cross-Validation

In [ ]:
class DNN_Network(nn.Module):
    def __init__(self):
        super(DNN_Network, self).__init__()
        self.encode1 = nn.Linear(13, 96)
        #self.encode2 = nn.Linear(128, 96)
        self.encode3 = nn.Linear(96, 64)
        self.encode4 = nn.Linear(64, 32)
        self.last_encode = nn.Linear(32, 16)
        #self.latent_layer = nn.Linear(16,16)
        self.decode4 = nn.Linear(16, 32)
        self.decode3 = nn.Linear(32, 64)
        #self.decode2 = nn.Linear(64, 96)
        self.decode1 = nn.Linear(64, 96)
        self.last_decode = nn.Linear(96, 13)
        self.output = nn.Linear(13, 1)
        self.dropout = 0.1
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        encode1 = F.relu(self.encode1(x))
        #encode2 = F.relu(self.encode2(encode1))
        encode3 = F.relu(self.encode3(encode1))
        encode4 = F.relu(self.encode4(encode3))
        last_encode = F.relu(self.last_encode(encode4))
        #latent_layer = F.relu(self.latent_layer(last_encode))
        decode4 = F.relu(self.decode4(last_encode)  + encode4)
        decode3 = F.relu(self.decode3(decode4) + encode3)
        #decode2 = F.relu(self.decode2(decode3) + encode2)
        decode1 = F.relu(self.decode1(decode3) + encode1)
        last_decode = F.relu(self.last_decode(decode1) + x)
        output = self.output(last_decode)
        return output

In [ ]:
net = NeuralNetRegressor(
    DNN_Network,
    #max_epochs = 3,
    #lr = 0.0004,
    #optimizer=optim.Adam,
    #iterator_train__shuffle=True,
    #iterator_train__num_workers=4,
    #iterator_valid__shuffle=True,
    #iterator_valid__num_workers=4,
    #optimizer__momentum = 0.5,
    batch_size = 256,
    #train_split = ,
    #callbacks=[EarlyStopping(patience=50)],
    device = 'cuda'
)

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {
    'lr': [0.0001,0.0002],
    'max_epochs': [1,2]
    #'module__num_units': [10, 20],
}


In [ ]:
gs = GridSearchCV(net, params, cv=5, scoring='accuracy')
gs.fit(train_x, train_y)
print(gs.best_score_, gs.best_params_)

In [ ]:
params = {
    'lr': [0.0001, 0.0002],
    'max_epochs': [2, 4],
    #'module__num_units': [10, 20],
}
gs = GridSearchCV(net, params, refit=False, cv=3, scoring='accuracy')

gs.fit(train_x, train_y)
print(gs.best_score_, gs.best_params_)

# (Pytorch) 1. 일반적인 학습방법

In [ ]:
class DNN_Network(nn.Module):
    def __init__(self):
        super(DNN_Network, self).__init__()
        self.encode1 = nn.Linear(13, 96)
        #self.encode2 = nn.Linear(128, 96)
        self.encode3 = nn.Linear(96, 64)
        self.encode4 = nn.Linear(64, 32)
        self.last_encode = nn.Linear(32, 16)
        #self.latent_layer = nn.Linear(16,16)
        self.decode4 = nn.Linear(16, 32)
        self.decode3 = nn.Linear(32, 64)
        #self.decode2 = nn.Linear(64, 96)
        self.decode1 = nn.Linear(64, 96)
        self.last_decode = nn.Linear(96, 13)
        self.output = nn.Linear(13, 1)
        #self.dropout = 0.1
        #self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        encode1 = F.relu(self.encode1(x))
        #encode2 = F.relu(self.encode2(encode1))
        encode3 = F.relu(self.encode3(encode1))
        encode4 = F.relu(self.encode4(encode3))
        last_encode = F.relu(self.last_encode(encode4))
        #latent_layer = F.relu(self.latent_layer(last_encode))
        decode4 = F.relu(self.decode4(last_encode)  + encode4)
        decode3 = F.relu(self.decode3(decode4) + encode3)
        #decode2 = F.relu(self.decode2(decode3) + encode2)
        decode1 = F.relu(self.decode1(decode3) + encode1)
        last_decode = F.relu(self.last_decode(decode1) + x)
        output = self.output(last_decode)
        return output

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
class CustomDataset(Dataset): 
    def __init__(self,input_data, target_data):
        self.x_data = input_data
        self.y_data = target_data

    # 총 데이터의 개수를 리턴
    def __len__(self): 
        return len(self.x_data)

    # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
    def __getitem__(self, idx): 
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return x, y

In [ ]:
torch_train_x = torch.tensor(train_x)
torch_train_y = torch.tensor(train_y)
torch_test_x = torch.tensor(test_x)
torch_test_y = torch.tensor(test_y)

In [ ]:
kfold = KFold(n_splits = 5, shuffle = False)
print(torch_train_x.shape, torch_train_y.shape, torch_test_x.shape, torch_test_y.shape)

torch.Size([15955, 13]) torch.Size([15955, 1]) torch.Size([6839, 13]) torch.Size([6839, 1])


In [ ]:
for train_index, test_index in kfold.split(torch_train_x, torch_train_y):  
    print(train_index, test_index)

    x_train_fold = torch_train_x[train_index] 
    y_train_fold = torch_train_y[train_index] 
    x_test_fold = torch_train_x[test_index] 
    y_test_fold = torch_train_y[test_index] 

    print(x_train_fold.shape, y_train_fold.shape) 
    print(x_test_fold.shape, y_test_fold.shape) 
    break 

[ 3191  3192  3193 ... 15952 15953 15954] [   0    1    2 ... 3188 3189 3190]
torch.Size([12764, 13]) torch.Size([12764, 1])
torch.Size([3191, 13]) torch.Size([3191, 1])


In [ ]:
seed = 1
lr = 0.0004
momentum = 0.5
no_cuda = False
log_interval = 200
split = 4
batch_size = 128
epochs = 10

In [ ]:
torch.manual_seed(seed)
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
model = DNN_Network().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = torch.nn.MSELoss(reduction='sum')

In [ ]:
def train(network, epochs, save_Model = False):
    total_acc = 0
    for fold, (train_index, test_index) in enumerate(kfold.split(train_x, train_y)):
        ### Dividing data into folds
        x_train_fold = torch_train_x[train_index]
        x_test_fold = torch_train_x[test_index]
        y_train_fold = torch_train_y[train_index]
        y_test_fold = torch_train_y[test_index]

        train = torch.utils.data.TensorDataset(x_train_fold, y_train_fold)
        test = torch.utils.data.TensorDataset(x_test_fold, y_test_fold)
        train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)
        test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False)

        for epoch in range(epochs):
            print('\nEpoch {} / {} \nFold number {} / {}'.format(epoch + 1, epochs, fold + 1 , kfold.get_n_splits()))
            pred_r2_score = 0
            #network.train()
            for batch_index, (x_batch, y_batch) in enumerate(train_loader):
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                out = network(x_batch)
                loss = loss_fn(out, y_batch)
                loss.backward()
                optimizer.step()
                validation_out = network
                #pred = torch.max(out.data, dim=1)[1]
                #pred = np.expand_dims(pred.cpu().detach().numpy(), axis =1)
                #pred_r2_score += r2_score(pred, y_batch.cpu().detach().numpy()) 
                if (batch_index + 1) % 32 == 0:
                    print('[{}/{} ({:.0f}%)]\tLoss: {:.6f}\t R2_score:{:.3f}'.format(
                        (batch_index + 1)*len(x_batch), len(train_loader.dataset),
                        100.*batch_index / len(train_loader), loss.data,
                         r2_score(out.cpu().detach().numpy(),y_batch.cpu().detach().numpy())))
            #print('pred_r2_score : ',pred_r2_score)
            #print(pred.shape)
        #total_acc += pred_r2_score
    total_acc = (total_acc / kfold.get_n_splits())
    print('\n\n Validation R2_Score : {:.3f}%'.format(total_acc))

In [ ]:
def return_corr():
    r2_pred = model(torch_test_x.cuda()).cpu().detach().numpy()
    r2_target = np.array(torch_test_y)
    df_r2 = pd.DataFrame()
    df_r2['r2_pred'] = r2_pred[:,0]
    df_r2['r2_target'] = r2_target
    return df_r2.corr().iloc[0,1]

In [ ]:
def train(network, epochs, save_Model = False):
    max_test_r2_score = 0
    max_test_r_score = 0
    total_acc = 0
    for fold, (train_index, test_index) in enumerate(kfold.split(train_x, train_y)):
        ### Dividing data into folds
        x_train_fold = torch_train_x[train_index]
        x_test_fold = torch_train_x[test_index]
        y_train_fold = torch_train_y[train_index]
        y_test_fold = torch_train_y[test_index]

        train = torch.utils.data.TensorDataset(x_train_fold, y_train_fold)
        test = torch.utils.data.TensorDataset(x_test_fold, y_test_fold)
        train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)
        test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False)

        for epoch in range(epochs):
            print('\nEpoch {} / {} \nFold number {} / {}'.format(epoch + 1, epochs, fold + 1 , kfold.get_n_splits()))
            #network.train()
            for batch_index, (x_batch, y_batch) in enumerate(train_loader):
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                #print(x_batch.shape)
                out = network(x_batch)
                loss = loss_fn(out, y_batch)
                loss.backward()
                optimizer.step()
                #pred = torch.max(out.data, dim=1)[1]
                #pred = np.expand_dims(pred.cpu().detach().numpy(), axis =1)
                #pred_r2_score += r2_score(pred, y_batch.cpu().detach().numpy())
                if (batch_index + 1) % 100 == 0:
                    print('[{}/{} ({:.0f}%)]\tLoss: {:.6f}\t R2_score:{:.3f}'.format(
                        (batch_index + 1)*len(x_batch), len(train_loader.dataset),
                        100.*batch_index / len(train_loader), loss.data,
                         r2_score(out.cpu().detach().numpy(),y_batch.cpu().detach().numpy())/len(train_loader)))
                    #print(out.cpu().detach().numpy().shape, y_batch.cpu().detach().numpy().shape)
                    
            test_r2_score = r2_score(model(torch_test_x.cuda()).cpu().detach().numpy(), np.array(torch_test_y))
            test_r_score = return_corr()
            print("test r2 score : {:.3f}, test r score : {:.3f}".format(test_r2_score, test_r_score))
            if max_test_r2_score < test_r2_score:
                max_test_r2_score = test_r2_score
            if max_test_r_score < test_r_score:
                max_test_r_score = test_r_score
            #print('pred_r2_score : ',pred_r2_score)
        vali_r2_score = 0
        for vali_x_batch, vali_y_batch in test_loader:
            #print(vali_x_batch.shape, vali_y_batch.shape)
            vali_x_batch = vali_x_batch.to(device)
            vali_out = model2(vali_x_batch)
            r2 = r2_score(vali_out.cpu().detach().numpy(),vali_y_batch)
            vali_r2_score += r2
            
        print("vali_r2_score is {:.3f}".format(vali_r2_score/len(test_loader)))
    print("max test r2 score : {}, max test r score : {}".format(max_test_r2_score,max_test_r_score))

train(model, 300)

In [ ]:
def return_corr():
    r2_pred = model(torch_test_x.cuda()).cpu().detach().numpy()
    r2_target = np.array(torch_test_y)
    df_r2 = pd.DataFrame()
    df_r2['r2_pred'] = r2_pred[:,0]
    df_r2['r2_target'] = r2_target
    print(df_r2.corr().iloc[0,1])

# (Pytorch) 2. DNN

In [ ]:
class DNN_Network2(nn.Module):
    def __init__(self):
        super(DNN_Network2, self).__init__()
        self.fc1 = nn.Linear(13, 300)
        self.fc2 = nn.Linear(300, 300)
        self.fc3 = nn.Linear(300, 100)
        self.fc4 = nn.Linear(100, 20)
        self.output = nn.Linear(20, 1)
        #self.dropout = 0.1
        #self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.output(x)
        return x

In [ ]:
seed = 1
lr = 0.0004
momentum = 0.5
no_cuda = False
log_interval = 200
split = 4
batch_size = 128
epochs = 10

In [ ]:
torch.manual_seed(seed)
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
model2 = DNN_Network2().to(device)
optimizer = optim.Adam(model2.parameters(), lr=lr)
loss_fn = torch.nn.MSELoss(reduction='sum')

In [ ]:
def return_corr():
    r2_pred = model2(torch_test_x.cuda()).cpu().detach().numpy()
    r2_target = np.array(torch_test_y)
    df_r2 = pd.DataFrame()
    df_r2['r2_pred'] = r2_pred[:,0]
    df_r2['r2_target'] = r2_target
    return df_r2.corr().iloc[0,1]

In [ ]:
seed = 1
lr = 0.0001
momentum = 0.5
no_cuda = False
log_interval = 200
split = 4
batch_size = 128
epochs = 10

In [ ]:
torch_train_x = torch.tensor(train_x)
torch_train_y = torch.tensor(train_y)
torch_test_x = torch.tensor(test_x)
torch_test_y = torch.tensor(test_y)

In [ ]:
kfold = KFold(n_splits = 5, shuffle = False)
#print(torch_train_x.shape, torch_train_y.shape, torch_test_x.shape, torch_test_y.shape)

In [ ]:
def train(network, epochs, save_Model = False):
    max_test_r2_score = 0
    max_test_r_score = 0
    total_acc = 0
    for fold, (train_index, test_index) in enumerate(kfold.split(train_x, train_y)):
        ### Dividing data into folds
        x_train_fold = torch_train_x[train_index]
        x_test_fold = torch_train_x[test_index]
        y_train_fold = torch_train_y[train_index]
        y_test_fold = torch_train_y[test_index]

        train = torch.utils.data.TensorDataset(x_train_fold, y_train_fold)
        test = torch.utils.data.TensorDataset(x_test_fold, y_test_fold)
        train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)
        test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False)

        for epoch in range(epochs):
            print('\nEpoch {} / {} \nFold number {} / {}'.format(epoch + 1, epochs, fold + 1 , kfold.get_n_splits()))
            #network.train()
            for batch_index, (x_batch, y_batch) in enumerate(train_loader):
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                #print(x_batch.shape)
                out = network(x_batch)
                loss = loss_fn(out, y_batch)
                loss.backward()
                optimizer.step()
                #pred = torch.max(out.data, dim=1)[1]
                #pred = np.expand_dims(pred.cpu().detach().numpy(), axis =1)
                #pred_r2_score += r2_score(pred, y_batch.cpu().detach().numpy())
                if (batch_index + 1) % 100 == 0:
                    print('[{}/{} ({:.0f}%)]\tLoss: {:.6f}\t R2_score:{:.3f}'.format(
                        (batch_index + 1)*len(x_batch), len(train_loader.dataset),
                        100.*batch_index / len(train_loader), loss.data,
                         r2_score(out.cpu().detach().numpy(),y_batch.cpu().detach().numpy())/len(train_loader)))
                    #print(out.cpu().detach().numpy().shape, y_batch.cpu().detach().numpy().shape)
                    
            test_r2_score = r2_score(model2(torch_test_x.cuda()).cpu().detach().numpy(), np.array(torch_test_y))
            test_r_score = return_corr()
            print("test r2 score : {:.3f}, test r score : {:.3f}".format(test_r2_score, test_r_score))
            if max_test_r2_score < test_r2_score:
                max_test_r2_score = test_r2_score
            if max_test_r_score < test_r_score:
                max_test_r_score = test_r_score
            #print('pred_r2_score : ',pred_r2_score)
        vali_r2_score = 0
        for vali_x_batch, vali_y_batch in test_loader:
            #print(vali_x_batch.shape, vali_y_batch.shape)
            vali_x_batch = vali_x_batch.to(device)
            vali_out = model2(vali_x_batch)
            r2 = r2_score(vali_out.cpu().detach().numpy(),vali_y_batch)
            vali_r2_score += r2
            
        print("vali_r2_score is {:.3f}".format(vali_r2_score/len(test_loader)))
    print("max test r2 score : {}, max test r score : {}".format(max_test_r2_score,max_test_r_score))

In [ ]:
train(model2, 300)


Epoch 1 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 62758.777344	 R2_score:-0.021
test r2 score : -2.297, test r score : 0.472

Epoch 2 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 55693.589844	 R2_score:-0.008
test r2 score : -0.794, test r score : 0.568

Epoch 3 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 52373.496094	 R2_score:-0.005
test r2 score : -0.534, test r score : 0.601

Epoch 4 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 49555.195312	 R2_score:-0.005
test r2 score : -0.436, test r score : 0.623

Epoch 5 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 46816.242188	 R2_score:-0.004
test r2 score : -0.365, test r score : 0.642

Epoch 6 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 44228.785156	 R2_score:-0.003
test r2 score : -0.300, test r score : 0.658

Epoch 7 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 42064.824219	 R2_score:-0.002
test r2 score : -0.241, test r score : 0.670

Epoch 8 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 403

[9200/12764 (99%)]	Loss: 16152.964844	 R2_score:0.007
test r2 score : 0.423, test r score : 0.752

Epoch 64 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 15973.683594	 R2_score:0.008
test r2 score : 0.426, test r score : 0.754

Epoch 65 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 15707.490234	 R2_score:0.008
test r2 score : 0.430, test r score : 0.754

Epoch 66 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 15570.223633	 R2_score:0.008
test r2 score : 0.432, test r score : 0.755

Epoch 67 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 15330.384766	 R2_score:0.008
test r2 score : 0.435, test r score : 0.756

Epoch 68 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 15272.356445	 R2_score:0.008
test r2 score : 0.437, test r score : 0.757

Epoch 69 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 15041.202148	 R2_score:0.008
test r2 score : 0.441, test r score : 0.757

Epoch 70 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 14796.919922	 R2_score:0.008
test r2 score :

[9200/12764 (99%)]	Loss: 10277.615234	 R2_score:0.008
test r2 score : 0.475, test r score : 0.766

Epoch 126 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 10109.324219	 R2_score:0.008
test r2 score : 0.471, test r score : 0.765

Epoch 127 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 9752.099609	 R2_score:0.008
test r2 score : 0.473, test r score : 0.765

Epoch 128 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 9511.872070	 R2_score:0.008
test r2 score : 0.462, test r score : 0.764

Epoch 129 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 8990.863281	 R2_score:0.009
test r2 score : 0.462, test r score : 0.764

Epoch 130 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 8469.635742	 R2_score:0.009
test r2 score : 0.475, test r score : 0.764

Epoch 131 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 8723.730469	 R2_score:0.009
test r2 score : 0.497, test r score : 0.766

Epoch 132 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 9394.526367	 R2_score:0.009
test r2 score 

[9200/12764 (99%)]	Loss: 5788.892578	 R2_score:0.009
test r2 score : 0.528, test r score : 0.772

Epoch 188 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 4823.314941	 R2_score:0.009
test r2 score : 0.535, test r score : 0.768

Epoch 189 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 5705.438477	 R2_score:0.009
test r2 score : 0.530, test r score : 0.772

Epoch 190 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 4616.707031	 R2_score:0.009
test r2 score : 0.534, test r score : 0.769

Epoch 191 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 5291.535156	 R2_score:0.009
test r2 score : 0.534, test r score : 0.773

Epoch 192 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 4604.075195	 R2_score:0.009
test r2 score : 0.533, test r score : 0.769

Epoch 193 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 5418.284180	 R2_score:0.009
test r2 score : 0.534, test r score : 0.772

Epoch 194 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 4582.332031	 R2_score:0.009
test r2 score : 

[9200/12764 (99%)]	Loss: 12598.673828	 R2_score:0.008
test r2 score : 0.396, test r score : 0.778

Epoch 250 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 9974.197266	 R2_score:0.008
test r2 score : 0.431, test r score : 0.778

Epoch 251 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 8590.948242	 R2_score:0.009
test r2 score : 0.468, test r score : 0.781

Epoch 252 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 7885.880859	 R2_score:0.009
test r2 score : 0.488, test r score : 0.783

Epoch 253 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 7362.619629	 R2_score:0.009
test r2 score : 0.494, test r score : 0.783

Epoch 254 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 6837.267090	 R2_score:0.009
test r2 score : 0.501, test r score : 0.783

Epoch 255 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 6242.692871	 R2_score:0.009
test r2 score : 0.507, test r score : 0.783

Epoch 256 / 300 
Fold number 1 / 5
[9200/12764 (99%)]	Loss: 5331.010742	 R2_score:0.009
test r2 score :

...

[9200/12764 (99%)]	Loss: 563.416016	 R2_score:0.010
test r2 score : 0.630, test r score : 0.818

Epoch 47 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 615.775391	 R2_score:0.010
test r2 score : 0.630, test r score : 0.819

Epoch 48 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 595.451294	 R2_score:0.010
test r2 score : 0.622, test r score : 0.819

Epoch 49 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 590.516846	 R2_score:0.010
test r2 score : 0.629, test r score : 0.819

Epoch 50 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 569.096802	 R2_score:0.010
test r2 score : 0.627, test r score : 0.818

Epoch 51 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 623.877380	 R2_score:0.010
test r2 score : 0.621, test r score : 0.818

Epoch 52 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 623.806885	 R2_score:0.010
test r2 score : 0.632, test r score : 0.818

Epoch 53 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 735.578613	 R2_score:0.010
test r2 score : 0.626, test r s

[9200/12764 (99%)]	Loss: 994.849976	 R2_score:0.010
test r2 score : 0.608, test r score : 0.815

Epoch 110 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 1159.338379	 R2_score:0.010
test r2 score : 0.600, test r score : 0.814

Epoch 111 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 956.016052	 R2_score:0.010
test r2 score : 0.599, test r score : 0.814

Epoch 112 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 1028.768921	 R2_score:0.010
test r2 score : 0.603, test r score : 0.814

Epoch 113 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 973.839722	 R2_score:0.010
test r2 score : 0.593, test r score : 0.813

Epoch 114 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 907.007812	 R2_score:0.010
test r2 score : 0.594, test r score : 0.813

Epoch 115 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 954.278809	 R2_score:0.010
test r2 score : 0.596, test r score : 0.814

Epoch 116 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 863.290771	 R2_score:0.010
test r2 score : 0.589,

[9200/12764 (99%)]	Loss: 462.690704	 R2_score:0.010
test r2 score : 0.623, test r score : 0.819

Epoch 172 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 568.209595	 R2_score:0.010
test r2 score : 0.611, test r score : 0.818

Epoch 173 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 468.584137	 R2_score:0.010
test r2 score : 0.617, test r score : 0.819

Epoch 174 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 397.585205	 R2_score:0.010
test r2 score : 0.609, test r score : 0.818

Epoch 175 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 376.736084	 R2_score:0.010
test r2 score : 0.610, test r score : 0.819

Epoch 176 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 289.355225	 R2_score:0.010
test r2 score : 0.611, test r score : 0.819

Epoch 177 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 271.616058	 R2_score:0.010
test r2 score : 0.611, test r score : 0.819

Epoch 178 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 289.260956	 R2_score:0.010
test r2 score : 0.607, t

[9200/12764 (99%)]	Loss: 604.482178	 R2_score:0.010
test r2 score : 0.561, test r score : 0.814

Epoch 235 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 730.706055	 R2_score:0.010
test r2 score : 0.566, test r score : 0.814

Epoch 236 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 810.367493	 R2_score:0.010
test r2 score : 0.542, test r score : 0.813

Epoch 237 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 709.495239	 R2_score:0.010
test r2 score : 0.533, test r score : 0.811

Epoch 238 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 800.508850	 R2_score:0.010
test r2 score : 0.542, test r score : 0.813

Epoch 239 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 990.582764	 R2_score:0.010
test r2 score : 0.497, test r score : 0.809

Epoch 240 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 752.631226	 R2_score:0.010
test r2 score : 0.531, test r score : 0.812

Epoch 241 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 635.052246	 R2_score:0.010
test r2 score : 0.502, t

[9200/12764 (99%)]	Loss: 650.731323	 R2_score:0.010
test r2 score : 0.594, test r score : 0.815

Epoch 298 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 536.718628	 R2_score:0.010
test r2 score : 0.606, test r score : 0.817

Epoch 299 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 471.767578	 R2_score:0.010
test r2 score : 0.600, test r score : 0.816

Epoch 300 / 300 
Fold number 5 / 5
[9200/12764 (99%)]	Loss: 490.819275	 R2_score:0.010
test r2 score : 0.609, test r score : 0.817
vali_r2_score is 0.875
max test r2 score : 0.6389428705683989, max test r score : 0.823830281898587


In [ ]:
y_pred = net.predict(test_x)
y_pred
y_target = test_y
y_target
df = pd.DataFrame()
df['Pred'] = y_pred[:,-1]
df['Target'] = y_target[:,-1]
df.corr()

In [ ]:
print(r2_score(df['Pred'], df['Target']), mean_squared_error(df['Pred'], df['Target'])**0.5)

In [ ]:
y_result = net.predict(train_x)
train_target = train_y
train_df = pd.DataFrame()
train_df['Train'] = y_result[:,-1]
train_df['Target'] = train_target[:,-1]
train_df.corr()

In [ ]:
print(r2_score(train_df['Train'], train_df['Target']), mean_squared_error(train_df['Train'], train_df['Target'])**0.5)

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from skorch import NeuralNetClassifier
from skorch.callbacks import EarlyStopping
from skorch.dataset import Dataset
from skorch.toy import make_classifier
from sklearn.model_selection import cross_validate
import torch

In [ ]:
ds = Dataset(train_x, train_y)

In [ ]:
y = np.array([y for _, y in iter(ds)])

In [ ]:
class DNN_Network(nn.Module):
    def __init__(self):
        super(DNN_Network, self).__init__()
        self.fc1 = nn.Linear(13, 300)
        self.fc2 = nn.Linear(300, 300)
        self.fc3 = nn.Linear(300, 100)
        self.fc4 = nn.Linear(100, 20)
        self.output = nn.Linear(20, 1)
        self.dropout = 0.1
        self.dropout = nn.Dropout(0.2)

    def forward(self, x, **kwargs):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.output(x)
        return x

In [ ]:
net = NeuralNetRegressor(
    DNN_Network,
    optimizer=optim.SGD,
    max_epochs = 200,
    lr = 0.0001,
    optimizer__momentum=0.9,
    batch_size = 32,
    iterator_train__shuffle=True,
    iterator_train__num_workers=4,
    iterator_valid__shuffle=True,
    iterator_valid__num_workers=4,
    callbacks=[EarlyStopping(patience=10)],
    device='cuda'
)

In [ ]:
net.fit(train_x, train_y)

In [ ]:
y_pred = net.predict(test_x)
y_pred
y_target = test_y
y_target
df = pd.DataFrame()
df['Pred'] = y_pred[:,-1]
df['Target'] = y_target[:,-1]
df.corr()

In [ ]:
y_result = net.predict(train_x)
train_target = train_y
train_df = pd.DataFrame()
train_df['Train'] = y_result[:,-1]
train_df['Target'] = train_target[:,-1]
train_df.corr()